In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [2]:
conn = engine.connect()

In [3]:
#for quick reference
t = engine.table_names()
print(t)
book_col = conn.execute('SELECT * FROM Bookings;')
fac_col = conn.execute('SELECT * FROM Facilities;')
mem_col = conn.execute('SELECT * FROM Members;')
print(book_col.keys())
print(fac_col.keys())
print(mem_col.keys())

['Bookings', 'Facilities', 'Members']
['bookid', 'facid', 'memid', 'starttime', 'slots']
['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']


In [4]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!

qx = conn.execute('SELECT f.name, ((f.membercost * memrev) + (f.guestcost * guestrev)) AS revenue FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid INNER JOIN (SELECT facid, COUNT(facid) AS memrev FROM Bookings WHERE memid <> 0 GROUP BY facid) AS mr ON b.facid = mr.facid INNER JOIN (SELECT facid, COUNT(facid) AS guestrev FROM Bookings WHERE memid = 0 GROUP BY facid) AS gr ON b.facid = gr.facid GROUP BY f.name HAVING revenue > 1000 ORDER BY revenue')
                  
qx.fetchall()

[('Tennis Court 1', 4040),
 ('Tennis Court 2', 4205),
 ('Squash Court', 4970.0),
 ('Massage Room 2', 6987.3),
 ('Massage Room 1', 20807.9)]

In [5]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

qxi = conn.execute('SELECT m.firstname, m.surname, (r.firstname || " " || r.surname) AS recommended_by FROM Members AS m INNER JOIN (SELECT memid, firstname, surname FROM Members) AS r ON r.memid = m.recommendedby ORDER BY m.surname, m.firstname')

qxi.fetchall()

[('Florence', 'Bader', 'Ponder Stibbons'),
 ('Anne', 'Baker', 'Ponder Stibbons'),
 ('Timothy', 'Baker', 'Jemima Farrell'),
 ('Tim', 'Boothe', 'Tim Rownam'),
 ('Gerald', 'Butters', 'Darren Smith'),
 ('Joan', 'Coplin', 'Timothy Baker'),
 ('Erica', 'Crumpet', 'Tracy Smith'),
 ('Nancy', 'Dare', 'Janice Joplette'),
 ('Matthew', 'Genting', 'Gerald Butters'),
 ('John', 'Hunt', 'Millicent Purview'),
 ('David', 'Jones', 'Janice Joplette'),
 ('Douglas', 'Jones', 'David Jones'),
 ('Janice', 'Joplette', 'Darren Smith'),
 ('Anna', 'Mackenzie', 'Darren Smith'),
 ('Charles', 'Owen', 'Darren Smith'),
 ('David', 'Pinker', 'Jemima Farrell'),
 ('Millicent', 'Purview', 'Tracy Smith'),
 ('Henrietta', 'Rumney', 'Matthew Genting'),
 ('Ramnaresh', 'Sarwin', 'Florence Bader'),
 ('Jack', 'Smith', 'Darren Smith'),
 ('Ponder', 'Stibbons', 'Burton Tracy'),
 ('Henry', 'Worthington-Smyth', 'Tracy Smith')]

In [6]:
#Q12: Find the facilities with their usage by member, but not guests
qxii = conn.execute('SELECT f.name, COUNT(bookid) AS usage FROM Bookings AS b INNER JOIN Facilities AS f ON f.facid=b.facid WHERE memid <> 0 GROUP BY f.name ORDER BY usage')

qxii.fetchall()

[('Massage Room 2', 27),
 ('Squash Court', 195),
 ('Tennis Court 2', 276),
 ('Tennis Court 1', 308),
 ('Badminton Court', 344),
 ('Table Tennis', 385),
 ('Massage Room 1', 421),
 ('Snooker Table', 421),
 ('Pool Table', 783)]

In [7]:
#Q13: Find the facilities usage by month, but not guests
qxiii = conn.execute('SELECT f.name, COUNT(bookid) AS usage, CASE WHEN strftime("%m", starttime) = "07" THEN "JULY" WHEN strftime("%m", starttime) = "08" THEN "AUGUST"  WHEN strftime("%m", starttime) = "09" THEN "SEPTEMBER" END AS month FROM Bookings AS b INNER JOIN Facilities AS f ON f.facid=b.facid WHERE memid <> 0 GROUP BY month, f.name ORDER BY strftime("%m", starttime), usage')

qxiii.fetchall()

[('Massage Room 2', 4, 'JULY'),
 ('Squash Court', 23, 'JULY'),
 ('Tennis Court 2', 41, 'JULY'),
 ('Table Tennis', 48, 'JULY'),
 ('Badminton Court', 51, 'JULY'),
 ('Tennis Court 1', 65, 'JULY'),
 ('Snooker Table', 68, 'JULY'),
 ('Massage Room 1', 77, 'JULY'),
 ('Pool Table', 103, 'JULY'),
 ('Massage Room 2', 9, 'AUGUST'),
 ('Squash Court', 85, 'AUGUST'),
 ('Tennis Court 2', 109, 'AUGUST'),
 ('Tennis Court 1', 111, 'AUGUST'),
 ('Badminton Court', 132, 'AUGUST'),
 ('Table Tennis', 143, 'AUGUST'),
 ('Massage Room 1', 153, 'AUGUST'),
 ('Snooker Table', 154, 'AUGUST'),
 ('Pool Table', 272, 'AUGUST'),
 ('Massage Room 2', 14, 'SEPTEMBER'),
 ('Squash Court', 87, 'SEPTEMBER'),
 ('Tennis Court 2', 126, 'SEPTEMBER'),
 ('Tennis Court 1', 132, 'SEPTEMBER'),
 ('Badminton Court', 161, 'SEPTEMBER'),
 ('Massage Room 1', 191, 'SEPTEMBER'),
 ('Table Tennis', 194, 'SEPTEMBER'),
 ('Snooker Table', 199, 'SEPTEMBER'),
 ('Pool Table', 408, 'SEPTEMBER')]

In [8]:
conn.close()